In [1]:
from sklearn.datasets import load_boston
import pandas as pd
data = pd.read_csv('/Users/max/Downloads/NETOLOGY/DVP/дз/data/aac_shelter_outcomes.csv')
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import datetime
from datetime import datetime
from datetime import timedelta

In [57]:
# 1) Сразу можно сказать, что удалить смело можно столбец animal_id это просто номер.
# 2) Создаю новый признак возраст животного в днях на основании дат. Удаляем лишние столбцы 'date_of_birth','datetime','monthyear','age_upon_outcome'
# 3) Также думаю, что признак name не повлияет на решение о животном. удаляем также и его.
# 4) Не нужен и outcome_subtype. Для нашей задачи не имеет значение куда попадет животное если его не принимает приют.

In [3]:
data.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


In [5]:
data = data[(data['outcome_type'] == 'Transfer') | (data['outcome_type'] ==  'Adoption')]
data.shape

(56611, 12)

In [6]:
data['outcome_type'].replace(['Transfer','Adoption'],[0,1], inplace=True)

In [7]:
data = data.drop('animal_id', axis=1)

In [8]:
def animal_age_days(data):
    birth, date = data['date_of_birth'], data['datetime']
    age_days = datetime.strptime(date, '%Y-%m-%dT%H:%M:%S') - datetime.strptime(birth, '%Y-%m-%dT%H:%M:%S')
    if age_days.days == 0: # - присваеваем возраст 1 день, если дата рождения животного совпадает с датой поступления в приют.
        return 1
    else:
        return age_days.days

In [9]:
data['animal_age_in_days'] = data.apply(animal_age_days, axis=1)

In [10]:
data = data.drop(['date_of_birth','datetime','monthyear'], axis=1)

In [11]:
data = data.drop('age_upon_outcome', axis=1)

In [12]:
data = data.drop('outcome_subtype', axis=1)

In [13]:
data.head()  

,animal_type,breed,color,name,outcome_type,sex_upon_outcome,animal_age_in_days
0,Cat,Domestic Shorthair Mix,Orange Tabby,NaN,0,Intact Male,15
1,Dog,Beagle Mix,White/Brown,Lucy,0,Spayed Female,366
2,Dog,Pit Bull,Blue/White,*Johnny,1,Neutered Male,429
3,Dog,Miniature Schnauzer Mix,White,Monday,0,Neutered Male,3300
5,Dog,Leonberger Mix,Brown/White,*Edgar,0,Intact Male,126


In [14]:
min(data['animal_age_in_days'])

-123

In [15]:
data.sort_values(by='animal_age_in_days').head(10)

,animal_type,breed,color,name,outcome_type,sex_upon_outcome,animal_age_in_days
19814,Dog,Labrador Retriever Mix,Black/White,Sadie,0,Spayed Female,-123
64745,Cat,Domestic Shorthair Mix,Black,Penelope,1,Spayed Female,-97
22804,Dog,Border Collie Mix,Black/White,Juan,1,Neutered Male,-93
52977,Cat,Domestic Shorthair Mix,Orange Tabby,NaN,0,Intact Male,-24
45764,Cat,Domestic Shorthair Mix,White/Orange,Jellybean,1,Intact Female,-10
50533,Cat,Domestic Shorthair Mix,Tortie,NaN,0,Unknown,-1
16754,Cat,Domestic Shorthair Mix,Blue Cream,NaN,0,Intact Female,1
32388,Cat,Domestic Shorthair Mix,Brown Tabby/White,NaN,0,Intact Female,1
63655,Cat,Domestic Shorthair Mix,Orange Tabby,NaN,0,Unknown,1
9835,Dog,Beagle/Chihuahua Shorthair,White,NaN,0,Intact Female,1


Тут видим, что в некоторых случаях возраст получился отрицательным, то есть дата рождения была позже чем животное попало в приют. Такого не может быть. Это выброс. Удаляем.

In [16]:
data = data.query('animal_age_in_days > 0')

In [17]:
data.head()

,animal_type,breed,color,name,outcome_type,sex_upon_outcome,animal_age_in_days
0,Cat,Domestic Shorthair Mix,Orange Tabby,NaN,0,Intact Male,15
1,Dog,Beagle Mix,White/Brown,Lucy,0,Spayed Female,366
2,Dog,Pit Bull,Blue/White,*Johnny,1,Neutered Male,429
3,Dog,Miniature Schnauzer Mix,White,Monday,0,Neutered Male,3300
5,Dog,Leonberger Mix,Brown/White,*Edgar,0,Intact Male,126


In [18]:
data.shape

(56605, 7)

In [19]:
data.query('outcome_type == 0').shape

(23496, 7)

In [20]:
data.query('outcome_type == 1').shape

(33109, 7)

In [21]:
data = data.drop('name', axis=1)

In [22]:
data['animal_type'].unique()

array(['Cat', 'Dog', 'Other', 'Bird', 'Livestock'], dtype=object)

In [23]:
len(data['breed'].unique())

1803

здесь видим, что у нас 1803 уникальных породы животных, делать one hot encodding нецелесообразно. Поэтому удаляем признак.

In [24]:
data = data.drop('breed', axis=1)

In [25]:
data.head()

,animal_type,color,outcome_type,sex_upon_outcome,animal_age_in_days
0,Cat,Orange Tabby,0,Intact Male,15
1,Dog,White/Brown,0,Spayed Female,366
2,Dog,Blue/White,1,Neutered Male,429
3,Dog,White,0,Neutered Male,3300
5,Dog,Brown/White,0,Intact Male,126


In [26]:
len(data['color'].unique())

475

цвета тоже очень много, удаляем и этот признак.

In [27]:
data = data.drop('color', axis=1)

In [28]:
data.head()

,animal_type,outcome_type,sex_upon_outcome,animal_age_in_days
0,Cat,0,Intact Male,15
1,Dog,0,Spayed Female,366
2,Dog,1,Neutered Male,429
3,Dog,0,Neutered Male,3300
5,Dog,0,Intact Male,126


In [29]:
data['sex_upon_outcome'].unique()

array(['Intact Male', 'Spayed Female', 'Neutered Male', 'Intact Female',
       'Unknown'], dtype=object)

Вижу, что есть значение в Unknown в признаке 'sex_upon_outcome', считаю какой процент строк с этим значением от общего числа в датасете. Получаю меньше 5ти процентов. В принципе это не так много, сл-но можем удалить строки со значением Unknown.

In [30]:
len(data[data['sex_upon_outcome']=='Unknown'])/len(data)*100

4.875894355622295

In [31]:
data = data[data['sex_upon_outcome']!='Unknown']

In [32]:
data[data['animal_type']=='Other'].shape # тип животных Other всего 485, удаляем

(485, 4)

In [33]:
data = data[data['animal_type']!='Other']

In [34]:
data.shape

(53360, 4)

In [35]:
import pandas_profiling

In [237]:
data.profile_report()

In [36]:
data.head()

,animal_type,outcome_type,sex_upon_outcome,animal_age_in_days
0,Cat,0,Intact Male,15
1,Dog,0,Spayed Female,366
2,Dog,1,Neutered Male,429
3,Dog,0,Neutered Male,3300
5,Dog,0,Intact Male,126


In [38]:
type_ = pd.get_dummies(data['animal_type'])

In [293]:
data.head()

,animal_type,outcome_subtype,outcome_type,sex_upon_outcome,animal_age_in_days
0,Cat,Partner,0,Intact Male,15
1,Dog,Partner,0,Spayed Female,366
2,Dog,Partner,1,Neutered Male,429
3,Dog,Partner,0,Neutered Male,3300
5,Dog,Partner,0,Intact Male,126


In [39]:
data = data.join(type_, how='left')

In [40]:
data.head()

,animal_type,outcome_type,sex_upon_outcome,animal_age_in_days,Bird,Cat,Dog,Livestock
0,Cat,0,Intact Male,15,0,1,0,0
1,Dog,0,Spayed Female,366,0,0,1,0
2,Dog,1,Neutered Male,429,0,0,1,0
3,Dog,0,Neutered Male,3300,0,0,1,0
5,Dog,0,Intact Male,126,0,0,1,0


In [41]:
data = data.drop('animal_type', axis=1)

In [42]:
sex = pd.get_dummies(data['sex_upon_outcome'])

In [43]:
data = pd.concat([data,sex],axis=1)

In [44]:
data.head()

,outcome_type,sex_upon_outcome,animal_age_in_days,Bird,Cat,Dog,Livestock,Intact Female,Intact Male,Neutered Male,Spayed Female
0,0,Intact Male,15,0,1,0,0,0,1,0,0
1,0,Spayed Female,366,0,0,1,0,0,0,0,1
2,1,Neutered Male,429,0,0,1,0,0,0,1,0
3,0,Neutered Male,3300,0,0,1,0,0,0,1,0
5,0,Intact Male,126,0,0,1,0,0,1,0,0


In [45]:
data = data.drop('sex_upon_outcome', axis=1)

In [46]:
data.head()

,outcome_type,animal_age_in_days,Bird,Cat,Dog,Livestock,Intact Female,Intact Male,Neutered Male,Spayed Female
0,0,15,0,1,0,0,0,1,0,0
1,0,366,0,0,1,0,0,0,0,1
2,1,429,0,0,1,0,0,0,1,0
3,0,3300,0,0,1,0,0,0,1,0
5,0,126,0,0,1,0,0,1,0,0


In [50]:
data.head()

,outcome_type,animal_age_in_days,Bird,Cat,Dog,Livestock,Intact Female,Intact Male,Neutered Male,Spayed Female
0,0,15,0,1,0,0,0,1,0,0
1,0,366,0,0,1,0,0,0,0,1
2,1,429,0,0,1,0,0,0,1,0
3,0,3300,0,0,1,0,0,0,1,0
5,0,126,0,0,1,0,0,1,0,0


In [51]:
X_columns = data.columns[1:]
Y_column = data.columns[0]
X_columns, Y_column

(Index(['animal_age_in_days', 'Bird', 'Cat', 'Dog', 'Livestock',
        'Intact Female', 'Intact Male', 'Neutered Male', 'Spayed Female'],
       dtype='object'), 'outcome_type')

outcome_type - target value(1 - adoption, 0 - transfer)

In [52]:
X = data[X_columns]
Y = data[Y_column]

In [53]:
from sklearn.model_selection import train_test_split
X_train,X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=33)

In [54]:
X_train.shape, X_test.shape

((42688, 9), (10672, 9))

In [55]:
from sklearn.ensemble import RandomForestClassifier

model=RandomForestClassifier(n_estimators=100)

model.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [56]:
model.score(X_test, Y_test)

0.7951649175412294